In [2]:
import scipy
import vqe_methods
import pyscf_helper

import pyscf
from pyscf import lib
from pyscf import gto, scf, mcscf, fci, ao2mo, lo, molden, cc
from pyscf.cc import ccsd

import openfermion as of
from openfermion import *
from tVQE import *

import numpy as np

In [3]:
geometry = [('Be', (0,0,0))]
n_act = 5
n_frz = 0
act_max = n_frz+n_act
shift = 2*n_frz
charge = 0
spin = 0
basis = 'cc-pvdz'

[n_orb, n_a, n_b, h, g, mol, E_nuc, E_scf, C, S] = pyscf_helper.init(geometry,charge,spin,basis,reference='rhf')
sq_ham = pyscf_helper.SQ_Hamiltonian()
sq_ham.init(h, g, C, S)
print(" HF Energy: %21.15f" %(E_nuc + sq_ham.energy_of_determinant(range(n_a),range(n_b))))

 ---------------------------------------------------------
                                                          
                      Using Pyscf:
                                                          
 ---------------------------------------------------------
                                                          


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = atom
damping factor = 0
level shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
DIIS start cycle = 1
DIIS space = 8
SCF tol = 1e-09
SCF gradient tol = 1e-14
max. SCF cycles = 1000
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/harjeet/research/downfolding-data/tmpaxruji_v
max_memory 8000 MB (current use 108 MB)
Initial guess from superpostion of atomic densties.
init E= -14.5723376309534
  HOMO = -0.309038559802575  LUMO = 0.0582587773072857
cycle= 1 E= -14.5723376309534  delta_E= -5.33e-15  |g|= 1.17e-08  |ddm|= 2.13e-07
  HOMO = -0.309038565850584  LUM

In [4]:
mf = scf.RHF(mol)
mf.conv_tol_grad = 1e-14
mf.max_cycle = 1000
mf.verbose = 4
mf.init_guess = 'atom'
mf.kernel()
print("MO energies:")
print(mf.mo_energy)

mccsd = cc.UCCSD(mf)
mccsd.conv_tol = 1e-12
mccsd.conv_tol_normt = 1e-10
#myccsd.diis_start_cycle=10000
mccsd.max_cycle = 1000
mccsd.kernel()
t1_amps = mccsd.t1
t2_amps = mccsd.t2

#print(t1_amps.shape)
#print(t2_amps.shape)



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = atom
damping factor = 0
level shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
DIIS start cycle = 1
DIIS space = 8
SCF tol = 1e-09
SCF gradient tol = 1e-14
max. SCF cycles = 1000
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/harjeet/research/downfolding-data/tmprbzw_ddk
max_memory 8000 MB (current use 126 MB)
Initial guess from superpostion of atomic densties.
init E= -14.5723376309534
  HOMO = -0.309038559802575  LUMO = 0.0582587773072857
cycle= 1 E= -14.5723376309534  delta_E= -5.33e-15  |g|= 1.17e-08  |ddm|= 2.13e-07
  HOMO = -0.309038565850584  LUMO = 0.0582587702364351
cycle= 2 E= -14.5723376309534  delta_E= 1.78e-15  |g|= 2.76e-09  |ddm|= 5.01e-08
  HOMO = -0.309038566563188  LUMO = 0.0582587694032785
cycle= 3 E= -14.5723376309534  delta_E= -1.78e-15  |g|= 1.7e-09  |ddm|= 5.91e-09
  HOMO = -0.309038566856709  LUMO = 0.0582587690601006
cycle= 4 E= -14.5723376309534  delt

In [5]:
#comment = """
# Calculating UCCSD energy from the t1 and t2 amps
UCCSD_corr = 0
for i in range(0,n_a):
    ia = 2*i
    ib = 2*i+1
    for j in range(0,n_b):
        ja = 2*j
        jb = 2*j+1
        for a in range(n_a,n_orb):
            aa = 2*a
            ab = 2*a+1
            for b in range(n_b,n_orb):
                ba = 2*b
                bb = 2*b+1 
                
                UCCSD_corr += 0.5 * t1_amps[0][i,a-n_a] * t1_amps[0][j,b-n_b] * (sq_ham.int_V[ia,aa,ja,ba] - sq_ham.int_V[ia,ba,ja,aa] +
                                                                                sq_ham.int_V[ib,ab,jb,bb] - sq_ham.int_V[ib,bb,jb,ab] +
                                                                                sq_ham.int_V[ia,aa,jb,bb] + sq_ham.int_V[ib,ab,ja,ba])
                
                UCCSD_corr += 0.5 * t1_amps[1][i,a-n_a] * t1_amps[1][j,b-n_b] * (sq_ham.int_V[ia,aa,ja,ba] - sq_ham.int_V[ia,ba,ja,aa] +
                                                                                sq_ham.int_V[ib,ab,jb,bb] - sq_ham.int_V[ib,bb,jb,ab] +
                                                                                sq_ham.int_V[ia,aa,jb,bb] + sq_ham.int_V[ib,ab,ja,ba])
                
                UCCSD_corr += 0.25 * t2_amps[0][i,j,a-n_a,b-n_b] * (sq_ham.int_V[ia,aa,ja,ba] - sq_ham.int_V[ia,ba,ja,aa] +
                                                                    sq_ham.int_V[ib,ab,jb,bb] - sq_ham.int_V[ib,bb,jb,ab] +
                                                                    sq_ham.int_V[ia,aa,jb,bb] + sq_ham.int_V[ib,ab,ja,ba])
                UCCSD_corr += 0.25 * t2_amps[1][i,j,a-n_a,b-n_b] * (sq_ham.int_V[ia,aa,ja,ba] - sq_ham.int_V[ia,ba,ja,aa] +
                                                                    sq_ham.int_V[ib,ab,jb,bb] - sq_ham.int_V[ib,bb,jb,ab] +
                                                                    sq_ham.int_V[ia,aa,jb,bb] + sq_ham.int_V[ib,ab,ja,ba])
                UCCSD_corr += 0.25 * t2_amps[2][i,j,a-n_a,b-n_b] *(sq_ham.int_V[ia,aa,ja,ba] - sq_ham.int_V[ia,ba,ja,aa] +
                                                                    sq_ham.int_V[ib,ab,jb,bb] - sq_ham.int_V[ib,bb,jb,ab] +
                                                                    sq_ham.int_V[ia,aa,jb,bb] + sq_ham.int_V[ib,ab,ja,ba])
                
print("UCCSD: %15.12f"%UCCSD_corr)
#"""

UCCSD: -0.022566791072


In [6]:
fmat = sq_ham.make_f(range(n_a),range(n_b))
vmat = sq_ham.make_v()
print(fmat.shape)
print(vmat.shape)
print(n_orb)
print(n_a)
print(n_b)

(28, 28)
(28, 28, 28, 28)
14
2
2


In [7]:
# getting t1_external_a               
for i in range(0,n_a):
    for a in range(n_a,act_max):
        t1_amps[0][i,a-n_a]=0
# getting t1_external_b               
for i in range(0,n_b):
    for a in range(n_b,act_max):
        t1_amps[1][i,a-n_b]=0
        
#print(t1_amps)


# getting t2_external_aa
for i in range(0,n_a):
    for j in range(0,n_a):
        for a in range(n_a,act_max):
            for b in range(n_a,act_max):
                t2_amps[0][i,j,a-n_a,b-n_a]=0

# getting t2_external_ab
for i in range(0,n_a):
    for j in range(0,n_b):
        for a in range(n_a,act_max):
            for b in range(n_b,act_max):
                t2_amps[1][i,j,a-n_a,b-n_b]=0 
                
# getting t2_external_bb
for i in range(0,n_b):
    for j in range(0,n_b):
        for a in range(n_b,act_max):
            for b in range(n_b,act_max):
                t2_amps[2][i,j,a-n_b,b-n_b]=0 
                

#print(t2_amps)

In [8]:
# expanding the t1 amplitude into alpha and beta space
t1 = np.zeros((n_a+n_b,2*n_orb-n_a-n_b))
for i in range(0,n_a):
    for a in range(n_a,n_orb):
        ia = 2*i
        aa = 2*a
        t1[ia,aa-(n_a+n_b)] = t1_amps[0][i,a-n_a]
for i in range(0,n_b):
    for a in range(n_b,n_orb):
        ib = 2*i+1
        ab = 2*a+1
        t1[ib,ab-(n_a+n_b)] = t1_amps[1][i,a-n_b]

print(t1.shape)
#print(t1)

# expanding the t2 amplitude_aa
t2 = np.zeros((n_a+n_b,n_a+n_b,2*n_orb-n_a-n_b,2*n_orb-n_a-n_b))
for i in range(0,n_a):
    ia = 2*i
    for j in range(0,n_a):
        ja = 2*j
        for a in range(n_a,n_orb):
            aa = 2*a
            for b in range(n_a,n_orb):
                ba = 2*b
                t2[ia,ja,aa-(n_a+n_b),ba-(n_a+n_b)]=t2_amps[0][i,j,a-n_a,b-n_a]

                
                
# expanding the t2 amplitude_ab
for i in range(0,n_a):
    ia = 2*i
    for j in range(0,n_b):
        jb = 2*j+1
        for a in range(n_a,n_orb):
            aa = 2*a
            for b in range(n_b,n_orb):
                bb = 2*b+1 
                t2[ia,jb,aa-(n_a+n_b),bb-(n_a+n_b)]=t2_amps[1][i,j,a-n_a,b-n_b]
                t2[jb,ia,bb-(n_a+n_b),aa-(n_a+n_b)]=t2_amps[1][i,j,a-n_a,b-n_b]
                t2[ia,jb,bb-(n_a+n_b),aa-(n_a+n_b)]=-t2_amps[1][i,j,a-n_a,b-n_b]
                t2[jb,ia,aa-(n_a+n_b),bb-(n_a+n_b)]=-t2_amps[1][i,j,a-n_a,b-n_b]
                
                

# expanding the t2 amplitude_bb
for i in range(0,n_b):
    ib = 2*i+1
    for j in range(0,n_b):
        jb = 2*j+1
        for a in range(n_b,n_orb):
            ab = 2*a+1
            for b in range(n_b,n_orb):
                bb = 2*b+1 
                t2[ib,jb,ab-(n_a+n_b),bb-(n_a+n_b)]=t2_amps[2][i,j,a-n_b,b-n_b]


print(t2.shape)                 

(4, 24)
(4, 4, 24, 24)


In [9]:
# Extracting different blocks of F
fmat_1 = np.array(fmat)

fmat_oo = fmat_1[0:n_a+n_b , 0:n_a+n_b]
print(fmat_oo.shape)

fmat_ov = fmat_1[0:n_a+n_b , n_a+n_b:2*n_orb]
print(fmat_ov.shape)

fmat_vo = fmat_1[n_a+n_b:2*n_orb , 0:n_a+n_b]
print(fmat_vo.shape)

fmat_vv = fmat_1[n_a+n_b:2*n_orb , n_a+n_b:2*n_orb]
print(fmat_vv.shape)

(4, 4)
(4, 24)
(24, 4)
(24, 24)


In [10]:
# FT1
# t_ov = t1 and t_vo = transpose(t1)

# f_oo and t1
ft1oo = 0
ft1oo += 1.000000000 * np.einsum("ai,ja->ji",fmat_vo,t1,optimize="optimal")
ft1oo += 1.000000000 * np.einsum("ia,aj->ij",fmat_ov,t1.transpose(),optimize="optimal")
print("ft1oo = ",ft1oo.shape)

# f_ov and t1
ft1ov = 0
ft1ov += -1.000000000 * np.einsum("ij,ja->ia",fmat_oo,t1,optimize="optimal")
ft1ov += 1.000000000 * np.einsum("ba,ib->ia",fmat_vv,t1,optimize="optimal")
print("ft1ov = ",ft1ov.shape)

# f_vo and t1
ft1vo = 0
ft1vo += -1.000000000 * np.einsum("ji,aj->ai",fmat_oo,t1.transpose(),optimize="optimal")
ft1vo += 1.000000000 * np.einsum("ab,bi->ai",fmat_vv,t1.transpose(),optimize="optimal")
print("ft1vo = ",ft1vo.shape)

# f_vv and t1
ft1vv = 0
ft1vv += -1.000000000 * np.einsum("ai,ib->ab",fmat_vo,t1,optimize="optimal")
ft1vv += -1.000000000 * np.einsum("ia,bi->ba",fmat_ov,t1.transpose(),optimize="optimal")
print("ft1vv = ",ft1vv.shape)

# f and t1
ft1 = 0
ft1 += 1.000000000 * np.einsum("ai,ia->",fmat_vo,t1,optimize="optimal")
ft1 += 1.000000000 * np.einsum("ia,ai->",fmat_ov,t1.transpose(),optimize="optimal")
print("ft1 = ",ft1)

# Joining all the sub-blocks to make ft1
ft1_mat = np.block([[ft1oo,ft1ov],[ft1vo,ft1vv]])
print("ft1_mat = ",ft1_mat.shape)

for i in range(0,2*n_orb):
    for a in range(0,2*n_orb):
        if abs(ft1_mat[i,a])>0.000000000000001:
            print("ft1_mat[",i,a,"]=",ft1_mat[i,a])
            
comment = """            
# Testing using openfermion
print("Making [FN,S1]")
fn_s1 = of.FermionOperator()
# adding terms from ft1_mat
for i in range(0,2*n_orb):
        for j in range(0,2*n_orb):
            if i<j:
                fn_s1 += -of.FermionOperator(((j,0),(i,0)), ft1_mat[i,j])
            elif i>j:
                fn_s1 += of.FermionOperator(((i,1),(j,1)), ft1_mat[i,j])
                
print(fn_s1)

"""


ft1oo =  (4, 4)
ft1ov =  (4, 24)
ft1vo =  (24, 4)
ft1vv =  (24, 24)
ft1 =  9.723002108282186e-17
ft1_mat =  (28, 28)
ft1_mat[ 0 10 ]= 0.001355329094271277
ft1_mat[ 1 11 ]= 0.0013553290942712793
ft1_mat[ 2 10 ]= 0.014042632662724786
ft1_mat[ 3 11 ]= 0.01404263266272478
ft1_mat[ 10 0 ]= 0.0013553290942712863
ft1_mat[ 10 2 ]= 0.014042632662724786
ft1_mat[ 11 1 ]= 0.0013553290942712884
ft1_mat[ 11 3 ]= 0.01404263266272478


In [11]:
# FT2
# t_oovv = t2 and t_vvoo = transpose(t2)

# Initializing the ft2_mat
ft2_mat = np.zeros((2*n_orb,2*n_orb,2*n_orb,2*n_orb))
print(ft2_mat.shape)

ft2ooov = 0
ft2ooov += -0.500000000 * np.einsum("bi,jkab->jkia",fmat_vo,t2,optimize="optimal")
#print(ft2ooov.shape)

for i in range(0,n_a+n_b):
    for j in range(0,n_a+n_b):
        for k in range(0,n_a+n_b):
            for a in range(0,2*n_orb-n_a-n_b):
                if abs(ft2ooov[i,j,k,a])>0.000000000000001:
                    print("ft2ooov[",i,j,k,a,"]=",ft2ooov[i,j,k,a])
                    ft2_mat[i,j,k,a+n_a+n_b] = ft2ooov[i,j,k,a]

ft2oovv = 0
ft2oovv += 0.500000000 * np.einsum("ik,jkab->ijab",fmat_oo,t2,optimize="optimal")
ft2oovv += -0.500000000 * np.einsum("ca,ijbc->ijab",fmat_vv,t2,optimize="optimal")
#print(ft2oovv.shape)


for i in range(0,n_a+n_b):
    for j in range(0,n_a+n_b):
        for a in range(0,2*n_orb-n_a-n_b):
            for b in range(0,2*n_orb-n_a-n_b):
                if abs(ft2oovv[i,j,a,b])>0.000000000000001:
                    print(f"ft2oovv[{i} {j} {a} {b}] = {ft2oovv[i,j,a,b]:.15f}")
                    ft2_mat[i,j,a+n_a+n_b,b+n_a+n_b] = ft2oovv[i,j,a,b]



ft2ovoo = 0
ft2ovoo += -0.500000000 * np.einsum("ib,abjk->iajk",fmat_ov,t2.transpose(),optimize="optimal")
#print(ft2ovoo.shape)

for i in range(0,n_a+n_b):
    for a in range(0,2*n_orb-n_a-n_b):
        for j in range(0,n_a+n_b):
            for k in range(0,n_a+n_b):
                if abs(ft2ovoo[i,a,j,k])>0.000000000000001:
                    print("ft2ovoo[",i,a,j,k,"]=",ft2ovoo[i,a,j,k])
                    ft2_mat[i,a+n_a+n_b,j,k] = ft2oovv[i,a,j,k]

ft2ovvv = 0
ft2ovvv += -0.500000000 * np.einsum("aj,ijbc->iabc",fmat_vo,t2,optimize="optimal")
#print(ft2ovvv.shape)

for i in range(0,n_a+n_b):
    for a in range(0,2*n_orb-n_a-n_b):
        for b in range(0,2*n_orb-n_a-n_b):
            for c in range(0,2*n_orb-n_a-n_b):
                if abs(ft2ovvv[i,a,b,c])>0.000000000000001:
                    print("ft2ovvv[",i,a,b,c,"]=",ft2ovvv[i,a,b,c])
                    ft2_mat[i,a+n_a+n_b,b+n_a+n_b,c+n_a+n_b] = ft2ovvv[i,a,b,c]
                    
ft2vvoo = 0
ft2vvoo += 0.500000000 * np.einsum("ki,abjk->abij",fmat_oo,t2.transpose(),optimize="optimal")
ft2vvoo += -0.500000000 * np.einsum("ac,bcij->abij",fmat_vv,t2.transpose(),optimize="optimal")
#print(ft2vvoo.shape)

for a in range(0,2*n_orb-n_a-n_b):
    for b in range(0,2*n_orb-n_a-n_b):
        for i in range(0,n_a+n_b):
            for j in range(0,n_a+n_b):
                if abs(ft2vvoo[a,b,i,j])>0.000000000000001:
                    print(f"ft2vvoo[{a} {b} {i} {j}] = {ft2vvoo[a,b,i,j]:.15f}")
                    ft2_mat[a+n_a+n_b,b+n_a+n_b,i,j] = ft2vvoo[a,b,i,j]

ft2vvov = 0
ft2vvov += -0.500000000 * np.einsum("ja,bcij->bcia",fmat_ov,t2.transpose(),optimize="optimal")
#print(ft2vvov.shape)

for a in range(0,2*n_orb-n_a-n_b):
    for b in range(0,2*n_orb-n_a-n_b):
        for i in range(0,n_a+n_b):
            for c in range(0,2*n_orb-n_a-n_b):
                if abs(ft2vvov[a,b,i,c])>0.000000000000001:
                    print("ft2vvov[",a,b,i,c,"]=",ft2vvov[a,b,i,c])
                    ft2_mat[a+n_a+n_b,b+n_a+n_b,i,c+n_a+n_b] = ft2vvoo[a,b,i,c]

                    
# printing the ft2_mat                
for p in range(0,2*n_orb):
    for q in range(0,2*n_orb):
        for r in range(0,2*n_orb):
            for s in range(0,2*n_orb):
                if abs(ft2_mat[p,q,r,s])>0.000000000000001:
                    print(p,q,r,s,ft2_mat[p,q,r,s])


ft2ov = 0
ft2ov += 1.000000000 * np.einsum("bj,ijab->ia",fmat_vo,t2,optimize="optimal")
#print(ft2ov.shape)

for i in range(0,n_a+n_b):
    for j in range(0,2*n_orb-n_a-n_b):
                if abs(ft2ov[i,j])>0.000000000000001:
                    print("ft2ov[",i,j,"]=",ft2ov[i,j])

ft2vo = 0
ft2vo += 1.000000000 * np.einsum("jb,abij->ai",fmat_ov,t2.transpose(),optimize="optimal")
#print(ft2vo.shape)

for i in range(0,2*n_orb-n_a-n_b):
    for j in range(0,n_a+n_b):
                if abs(ft2vo[i,j])>0.000000000000001:
                    print("ft2vo[",i,j,"]=",ft2vo[i,j])

                    
# Initializing the ft2_mat (two rank tensor)
ft2oo = np.zeros((n_a+n_b,n_a+n_b))
ft2vv = np.zeros((2*n_orb-n_a-n_b,2*n_orb-n_a-n_b))
print("Shape of ft2oo and ft2vv = ",ft2oo.shape ,ft2vv.shape)


# Joining all the sub-blocks to make ft2 (two rank tensor)
ft2_mat_1 = np.block([[ft2oo,ft2ov],[ft2vo,ft2vv]])
print("ft1_mat = ",ft2_mat_1.shape)
                    
                


comment = """
# Non zero elements in only ft2oovv and ft2vvoo
# Testing using openfermion
print("Making [FN,S2]")
fn_s2 = of.FermionOperator()
# adding terms from ft2oovv
for i in range(0,n_a+n_b):
    for j in range(0,n_a+n_b):
        for a in range(0,2*n_orb-n_a-n_b):
            for b in range(0,2*n_orb-n_a-n_b):
                fn_s2 += -of.FermionOperator(((a+n_a+n_b,0),(b+n_a+n_b,0),(i,0),(j,0)), ft2oovv[i,j,a,b])
                
# adding terms from ft2vvoo
for a in range(0,2*n_orb-n_a-n_b):
    for b in range(0,2*n_orb-n_a-n_b):
        for i in range(0,n_a+n_b):
            for j in range(0,n_a+n_b):
                fn_s2 += -of.FermionOperator(((a+n_a+n_b,1),(b+n_a+n_b,1),(i,1),(j,1)), ft2vvoo[a,b,i,j])
                
print(normal_ordered(fn_s2))
"""

(28, 28, 28, 28)
ft2oovv[0 1 0 9] = -0.001431681571797
ft2oovv[0 1 0 11] = 0.000346631260456
ft2oovv[0 1 1 8] = 0.001431681571797
ft2oovv[0 1 1 10] = -0.000346631260456
ft2oovv[0 1 2 9] = -0.000346631260456
ft2oovv[0 1 2 11] = -0.001431681571797
ft2oovv[0 1 3 8] = 0.000346631260456
ft2oovv[0 1 3 10] = 0.001431681571797
ft2oovv[0 1 4 13] = -0.001473046283641
ft2oovv[0 1 5 12] = 0.001473046283641
ft2oovv[0 1 6 7] = -0.003301625016294
ft2oovv[0 1 7 6] = 0.003301625016294
ft2oovv[0 1 8 1] = -0.001518923118786
ft2oovv[0 1 8 3] = -0.000367753727905
ft2oovv[0 1 8 9] = -0.004842786619746
ft2oovv[0 1 9 0] = 0.001518923118786
ft2oovv[0 1 9 2] = 0.000367753727905
ft2oovv[0 1 9 8] = 0.004842786619746
ft2oovv[0 1 10 1] = 0.000367753727905
ft2oovv[0 1 10 3] = -0.001518923118786
ft2oovv[0 1 10 11] = -0.004842786619746
ft2oovv[0 1 11 0] = -0.000367753727905
ft2oovv[0 1 11 2] = 0.001518923118786
ft2oovv[0 1 11 10] = 0.004842786619746
ft2oovv[0 1 12 5] = -0.001562808448010
ft2oovv[0 1 12 13] = -0.004842

ft2vvoo[0 8 0 2] = 0.001798915902739
ft2vvoo[0 8 2 0] = -0.000137924080533
ft2vvoo[0 9 0 1] = -0.001431681571797
ft2vvoo[0 9 0 3] = -0.002695788027525
ft2vvoo[0 9 1 0] = 0.001431681571797
ft2vvoo[0 9 1 2] = 0.004494703930264
ft2vvoo[0 9 2 1] = -0.000344611944286
ft2vvoo[0 9 2 3] = -0.012385280954545
ft2vvoo[0 9 3 0] = 0.000206687863753
ft2vvoo[0 9 3 2] = 0.012385280954545
ft2vvoo[0 10 0 2] = -0.000435544117565
ft2vvoo[0 10 2 0] = 0.000033393457613
ft2vvoo[0 11 0 1] = 0.000346631260456
ft2vvoo[0 11 0 3] = 0.000652690109527
ft2vvoo[0 11 1 0] = -0.000346631260456
ft2vvoo[0 11 1 2] = -0.001088234227092
ft2vvoo[0 11 2 1] = 0.000083435643071
ft2vvoo[0 11 2 3] = 0.002998659501493
ft2vvoo[0 11 3 0] = -0.000050042185459
ft2vvoo[0 11 3 2] = -0.002998659501493
ft2vvoo[1 8 0 1] = 0.001431681571797
ft2vvoo[1 8 0 3] = 0.004494703930264
ft2vvoo[1 8 1 0] = -0.001431681571797
ft2vvoo[1 8 1 2] = -0.002695788027525
ft2vvoo[1 8 2 1] = 0.000206687863753
ft2vvoo[1 8 2 3] = 0.012385280954545
ft2vvoo[1 8 3 0]

0 1 4 13 -0.0014316815717968568
0 1 4 15 0.00034663126045648995
0 1 5 12 0.0014316815717968577
0 1 5 14 -0.00034663126045649
0 1 6 13 -0.0003466312604564905
0 1 6 15 -0.0014316815717968583
0 1 7 12 0.0003466312604564906
0 1 7 14 0.0014316815717968583
0 1 8 17 -0.0014730462836409382
0 1 9 16 0.0014730462836409376
0 1 10 11 -0.0033016250162937123
0 1 11 10 0.0033016250162937123
0 1 12 5 -0.001518923118786365
0 1 12 7 -0.0003677537279051657
0 1 12 13 -0.004842786619746342
0 1 13 4 0.0015189231187863644
0 1 13 6 0.0003677537279051655
0 1 13 12 0.004842786619746342
0 1 14 5 0.0003677537279051651
0 1 14 7 -0.0015189231187863659
0 1 14 15 -0.004842786619746334
0 1 15 4 -0.0003677537279051651
0 1 15 6 0.0015189231187863659
0 1 15 14 0.004842786619746334
0 1 16 9 -0.0015628084480101344
0 1 16 17 -0.004842786619746333
0 1 17 8 0.0015628084480101352
0 1 17 16 0.004842786619746333
0 1 18 19 -3.6355018225304824e-05
0 1 19 18 3.6355018225304824e-05
0 1 20 21 -3.6355018225304885e-05
0 1 21 20 3.63550

3 2 5 14 0.0029986595014933294
3 2 6 13 0.0029986595014933307
3 2 6 15 0.012385280954544596
3 2 7 12 -0.00299865950149333
3 2 7 14 -0.012385280954544591
3 2 8 17 0.01274312140446371
3 2 9 16 -0.012743121404463709
3 2 10 11 0.011842641719296944
3 2 11 10 -0.011842641719296946
3 2 12 5 0.02222887170580786
3 2 12 7 0.005381938253377946
3 2 12 13 0.011311567114962178
3 2 13 4 -0.022228871705807868
3 2 13 6 -0.005381938253377947
3 2 13 12 -0.01131156711496218
3 2 14 5 -0.005381938253377952
3 2 14 7 0.02222887170580783
3 2 14 15 0.011311567114962156
3 2 15 4 0.005381938253377955
3 2 15 6 -0.02222887170580784
3 2 15 14 -0.011311567114962154
3 2 16 9 0.022871117084139896
3 2 16 17 0.011311567114962152
3 2 17 8 -0.0228711170841399
3 2 17 16 -0.011311567114962152
3 2 18 19 0.004132682752906768
3 2 19 18 -0.004132682752906768
3 2 20 21 0.004132682752906771
3 2 21 20 -0.004132682752906771
3 2 22 23 0.004132682752906783
3 2 23 22 -0.004132682752906783
3 2 24 25 0.0041326827529067816
3 2 25 24 -0.00

18 19 0 3 -0.000786770781614392
18 19 1 0 3.635501822530488e-05
18 19 1 2 0.0007867707816143914
18 19 2 1 -0.00014028199904406267
18 19 2 3 -0.0041326827529067686
18 19 3 0 0.00014028199904406272
18 19 3 2 0.004132682752906768
19 18 0 1 3.635501822530488e-05
19 18 0 3 0.000786770781614392
19 18 1 0 -3.635501822530488e-05
19 18 1 2 -0.0007867707816143914
19 18 2 1 0.00014028199904406267
19 18 2 3 0.0041326827529067686
19 18 3 0 -0.00014028199904406272
19 18 3 2 -0.004132682752906768
20 21 0 1 -3.635501822530494e-05
20 21 0 3 -0.00078677078161439
20 21 1 0 3.635501822530494e-05
20 21 1 2 0.0007867707816143899
20 21 2 1 -0.00014028199904406245
20 21 2 3 -0.004132682752906772
20 21 3 0 0.00014028199904406242
20 21 3 2 0.004132682752906771
21 20 0 1 3.635501822530494e-05
21 20 0 3 0.00078677078161439
21 20 1 0 -3.635501822530494e-05
21 20 1 2 -0.0007867707816143899
21 20 2 1 0.00014028199904406245
21 20 2 3 0.004132682752906772
21 20 3 0 -0.00014028199904406242
21 20 3 2 -0.0041326827529067

In [29]:
# Extracting different blocks of V

vmat_1 = np.array(vmat)
print(vmat_1.shape)

vmat_oooo = vmat_1[0:n_a+n_b , 0:n_a+n_b , 0:n_a+n_b , 0:n_a+n_b]
print(vmat_oooo.shape)

vmat_ooov = vmat_1[0:n_a+n_b , 0:n_a+n_b , 0:n_a+n_b , n_a+n_b:2*n_orb]
print(vmat_ooov.shape)

vmat_ovoo = vmat_1[0:n_a+n_b , n_a+n_b:2*n_orb , 0:n_a+n_b , 0:n_a+n_b]
print(vmat_ovoo.shape)

vmat_ovov = vmat_1[0:n_a+n_b , n_a+n_b:2*n_orb , 0:n_a+n_b , n_a+n_b:2*n_orb]
print(vmat_ovov.shape)

vmat_oovv = vmat_1[0:n_a+n_b , 0:n_a+n_b , n_a+n_b:2*n_orb , n_a+n_b:2*n_orb]
print(vmat_oovv.shape)

vmat_vvoo = vmat_1[n_a+n_b:2*n_orb , n_a+n_b:2*n_orb , 0:n_a+n_b , 0:n_a+n_b]
print(vmat_vvoo.shape)

vmat_ovvv = vmat_1[0:n_a+n_b , n_a+n_b:2*n_orb , n_a+n_b:2*n_orb , n_a+n_b:2*n_orb]
print(vmat_ovvv.shape)

vmat_vvov = vmat_1[n_a+n_b:2*n_orb , n_a+n_b:2*n_orb , 0:n_a+n_b , n_a+n_b:2*n_orb]
print(vmat_vvov.shape)

vmat_vvvv = vmat_1[n_a+n_b:2*n_orb , n_a+n_b:2*n_orb , n_a+n_b:2*n_orb , n_a+n_b:2*n_orb]
print(vmat_vvvv.shape)

(28, 28, 28, 28)
(4, 4, 4, 4)
(4, 4, 4, 24)
(4, 24, 4, 4)
(4, 24, 4, 24)
(4, 4, 24, 24)
(24, 24, 4, 4)
(4, 24, 24, 24)
(24, 24, 4, 24)
(24, 24, 24, 24)


In [30]:
# WT1
# t_ov = t1 and t_vo = transpose(t1)

wt1oooo = 0
wt1oooo += 0.500000000 * np.einsum("kaij,la->klij",vmat_ovoo,t1,optimize="optimal")
wt1oooo += 0.500000000 * np.einsum("jkia,al->jkil",vmat_ooov,t1.transpose(),optimize="optimal")
print("wt1oooo = ",wt1oooo.shape)

wt1ooov = 0
wt1ooov += -0.500000000 * np.einsum("jkil,la->jkia",vmat_oooo,t1,optimize="optimal")
wt1ooov += 1.000000000 * np.einsum("jbia,kb->jkia",vmat_ovov,t1,optimize="optimal")
wt1ooov += -0.500000000 * np.einsum("ijab,bk->ijka",vmat_oovv,t1.transpose(),optimize="optimal")
print("wt1ooov = ",wt1ooov.shape)

wt1oovv = 0
wt1oovv += 0.500000000 * np.einsum("ijka,kb->ijab",vmat_ooov,t1,optimize="optimal")
wt1oovv += 0.500000000 * np.einsum("icab,jc->ijab",vmat_ovvv,t1,optimize="optimal")
print("wt1oovv = ",wt1oovv.shape)

wt1ovoo = 0
wt1ovoo += -0.500000000 * np.einsum("klij,al->kaij",vmat_oooo,t1.transpose(),optimize="optimal")
wt1ovoo += -0.500000000 * np.einsum("abij,kb->kaij",vmat_vvoo,t1,optimize="optimal")
wt1ovoo += 1.000000000 * np.einsum("jaib,bk->jaik",vmat_ovov,t1.transpose(),optimize="optimal")
print("wt1ovoo = ",wt1ovoo.shape)

wt1ovov = 0
wt1ovov += -1.000000000 * np.einsum("jaik,kb->jaib",vmat_ovoo,t1,optimize="optimal")
wt1ovov += -1.000000000 * np.einsum("jkia,bk->jbia",vmat_ooov,t1.transpose(),optimize="optimal")
wt1ovov += -1.000000000 * np.einsum("bcia,jc->jbia",vmat_vvov,t1,optimize="optimal")
wt1ovov += -1.000000000 * np.einsum("ibac,cj->ibja",vmat_ovvv,t1.transpose(),optimize="optimal")
print("wt1ovov = ",wt1ovov.shape)

wt1ovvv = 0
wt1ovvv += 1.000000000 * np.einsum("ibja,jc->ibac",vmat_ovov,t1,optimize="optimal")
wt1ovvv += -0.500000000 * np.einsum("ijab,cj->icab",vmat_oovv,t1.transpose(),optimize="optimal")
wt1ovvv += -0.500000000 * np.einsum("cdab,id->icab",vmat_vvvv,t1,optimize="optimal")
print("wt1ovvv = ",wt1ovvv.shape)

wt1vvoo = 0
wt1vvoo += 0.500000000 * np.einsum("kaij,bk->abij",vmat_ovoo,t1.transpose(),optimize="optimal")
wt1vvoo += 0.500000000 * np.einsum("abic,cj->abij",vmat_vvov,t1.transpose(),optimize="optimal")
print("wt1vvoo = ",wt1vvoo.shape)

wt1vvov = 0
wt1vvov += -0.500000000 * np.einsum("abij,jc->abic",vmat_vvoo,t1,optimize="optimal")
wt1vvov += 1.000000000 * np.einsum("jbia,cj->bcia",vmat_ovov,t1.transpose(),optimize="optimal")
wt1vvov += -0.500000000 * np.einsum("bcad,di->bcia",vmat_vvvv,t1.transpose(),optimize="optimal")
print("wt1vvov = ",wt1vvov.shape)

wt1vvvv = 0
wt1vvvv += 0.500000000 * np.einsum("bcia,id->bcad",vmat_vvov,t1,optimize="optimal")
wt1vvvv += 0.500000000 * np.einsum("icab,di->cdab",vmat_ovvv,t1.transpose(),optimize="optimal")
print("wt1vvvv = ",wt1vvvv.shape)

wt1oo = 0
wt1oo += 1.000000000 * np.einsum("jaik,ka->ji",vmat_ovoo,t1,optimize="optimal")
wt1oo += 1.000000000 * np.einsum("jkia,ak->ji",vmat_ooov,t1.transpose(),optimize="optimal")
print("wt1oo = ",wt1oo.shape)

wt1ov = 0
wt1ov += -1.000000000 * np.einsum("ibja,jb->ia",vmat_ovov,t1,optimize="optimal")
wt1ov += 1.000000000 * np.einsum("ijab,bj->ia",vmat_oovv,t1.transpose(),optimize="optimal")
print("wt1ov = ",wt1ov.shape)

wt1vo = 0
wt1vo += 1.000000000 * np.einsum("abij,jb->ai",vmat_vvoo,t1,optimize="optimal")
wt1vo += -1.000000000 * np.einsum("jaib,bj->ai",vmat_ovov,t1.transpose(),optimize="optimal")
print("wt1vo = ",wt1vo.shape)

wt1vv = 0
wt1vv += -1.000000000 * np.einsum("bcia,ic->ba",vmat_vvov,t1,optimize="optimal")
wt1vv += -1.000000000 * np.einsum("ibac,ci->ba",vmat_ovvv,t1.transpose(),optimize="optimal")
print("wt1vv = ",wt1vv.shape)

wt1oooo =  (4, 4, 4, 4)
wt1ooov =  (4, 4, 4, 24)
wt1oovv =  (4, 4, 24, 24)
wt1ovoo =  (4, 24, 4, 4)
wt1ovov =  (4, 24, 4, 24)
wt1ovvv =  (4, 24, 24, 24)
wt1vvoo =  (24, 24, 4, 4)
wt1vvov =  (24, 24, 4, 24)
wt1vvvv =  (24, 24, 24, 24)
wt1oo =  (4, 4)
wt1ov =  (4, 24)
wt1vo =  (24, 4)
wt1vv =  (24, 24)


In [31]:
# WT2
# t_oovv = t2 and t_vvoo = transpose(t2)

wt2ooooov = 0
wt2ooooov += -0.250000000 * np.einsum("kbij,lmab->klmija",vmat_ovoo,t2,optimize="optimal")
print("wt2ooooov = ",wt2ooooov.shape)

wt2oooovv = 0
wt2oooovv += 0.250000000 * np.einsum("jkim,lmab->jkliab",vmat_oooo,t2,optimize="optimal")
wt2oooovv += -0.500000000 * np.einsum("jcia,klbc->jkliab",vmat_ovov,t2,optimize="optimal")
print("wt2oooovv = ",wt2oooovv.shape)

wt2ooovvv = 0
wt2ooovvv += -0.250000000 * np.einsum("ijla,klbc->ijkabc",vmat_ooov,t2,optimize="optimal")
wt2ooovvv += -0.250000000 * np.einsum("idab,jkcd->ijkabc",vmat_ovvv,t2,optimize="optimal")
print("wt2ooovvv = ",wt2ooovvv.shape)

wt2oovooo = 0
wt2oovooo += -0.250000000 * np.einsum("jkib,ablm->jkailm",vmat_ooov,t2.transpose(),optimize="optimal")
print("wt2oovooo = ",wt2oovooo.shape)

wt2oovoov = 0
wt2oovoov += -0.250000000 * np.einsum("acij,klbc->klaijb",vmat_vvoo,t2,optimize="optimal")
wt2oovoov += -0.250000000 * np.einsum("ijac,bckl->ijbkla",vmat_oovv,t2.transpose(),optimize="optimal")
print("wt2oovoov = ",wt2oovoov.shape)

wt2oovovv = 0
wt2oovovv += -0.500000000 * np.einsum("jail,klbc->jkaibc",vmat_ovoo,t2,optimize="optimal")
wt2oovovv += -0.500000000 * np.einsum("bdia,jkcd->jkbiac",vmat_vvov,t2,optimize="optimal")
print("wt2oovovv = ",wt2oovovv.shape)

wt2oovvvv = 0
wt2oovvvv += 0.500000000 * np.einsum("ibka,jkcd->ijbacd",vmat_ovov,t2,optimize="optimal")
wt2oovvvv += -0.250000000 * np.einsum("ceab,ijde->ijcabd",vmat_vvvv,t2,optimize="optimal")
print("wt2oovvvv = ",wt2oovvvv.shape)

wt2ovvooo = 0
wt2ovvooo += 0.250000000 * np.einsum("kmij,ablm->kabijl",vmat_oooo,t2.transpose(),optimize="optimal")
wt2ovvooo += -0.500000000 * np.einsum("jaic,bckl->jabikl",vmat_ovov,t2.transpose(),optimize="optimal")
print("wt2ovvooo = ",wt2ovvooo.shape)

wt2ovvoov = 0
wt2ovvoov += -0.500000000 * np.einsum("jlia,bckl->jbcika",vmat_ooov,t2.transpose(),optimize="optimal")
wt2ovvoov += -0.500000000 * np.einsum("ibad,cdjk->ibcjka",vmat_ovvv,t2.transpose(),optimize="optimal")
print("wt2ovvoov = ",wt2ovvoov.shape)

wt2ovvovv = 0
wt2ovvovv += 0.250000000 * np.einsum("abik,jkcd->jabicd",vmat_vvoo,t2,optimize="optimal")
wt2ovvovv += 0.250000000 * np.einsum("ikab,cdjk->icdjab",vmat_oovv,t2.transpose(),optimize="optimal")
print("wt2ovvovv = ",wt2ovvovv.shape)

wt2ovvvvv = 0
wt2ovvvvv += -0.250000000 * np.einsum("bcja,ijde->ibcade",vmat_vvov,t2,optimize="optimal")
print("wt2ovvvvv = ",wt2ovvvvv.shape)

wt2vvvooo = 0
wt2vvvooo += -0.250000000 * np.einsum("laij,bckl->abcijk",vmat_ovoo,t2.transpose(),optimize="optimal")
wt2vvvooo += -0.250000000 * np.einsum("abid,cdjk->abcijk",vmat_vvov,t2.transpose(),optimize="optimal")
print("wt2vvvooo = ",wt2vvvooo.shape)

wt2vvvoov = 0
wt2vvvoov += 0.500000000 * np.einsum("kbia,cdjk->bcdija",vmat_ovov,t2.transpose(),optimize="optimal")
wt2vvvoov += -0.250000000 * np.einsum("bcae,deij->bcdija",vmat_vvvv,t2.transpose(),optimize="optimal")
print("wt2vvvoov = ",wt2vvvoov.shape)

wt2vvvovv = 0
wt2vvvovv += -0.250000000 * np.einsum("jcab,deij->cdeiab",vmat_ovvv,t2.transpose(),optimize="optimal")
print("wt2vvvovv = ",wt2vvvovv.shape)

wt2oooo = 0
wt2oooo += 0.125000000 * np.einsum("abij,klab->klij",vmat_vvoo,t2,optimize="optimal")
wt2oooo += 0.125000000 * np.einsum("ijab,abkl->ijkl",vmat_oovv,t2.transpose(),optimize="optimal")
print("wt2oooo = ",wt2oooo.shape)

wt2ooov = 0
wt2ooov += 1.000000000 * np.einsum("jbil,klab->jkia",vmat_ovoo,t2,optimize="optimal")
wt2ooov += 0.250000000 * np.einsum("bcia,jkbc->jkia",vmat_vvov,t2,optimize="optimal")
print("wt2ooov = ",wt2ooov.shape)

wt2oovv = 0
wt2oovv += 0.125000000 * np.einsum("ijkl,klab->ijab",vmat_oooo,t2,optimize="optimal")
wt2oovv += -1.000000000 * np.einsum("icka,jkbc->ijab",vmat_ovov,t2,optimize="optimal")
wt2oovv += 0.125000000 * np.einsum("cdab,ijcd->ijab",vmat_vvvv,t2,optimize="optimal")
print("wt2oovv = ",wt2oovv.shape)

wt2ovoo = 0
wt2ovoo += 1.000000000 * np.einsum("jlib,abkl->jaik",vmat_ooov,t2.transpose(),optimize="optimal")
wt2ovoo += 0.250000000 * np.einsum("iabc,bcjk->iajk",vmat_ovvv,t2.transpose(),optimize="optimal")
print("wt2ovoo = ",wt2ovoo.shape)

wt2ovov = 0
wt2ovov += -1.000000000 * np.einsum("acik,jkbc->jaib",vmat_vvoo,t2,optimize="optimal")
wt2ovov += -1.000000000 * np.einsum("ikac,bcjk->ibja",vmat_oovv,t2.transpose(),optimize="optimal")
print("wt2ovov = ",wt2ovov.shape)

wt2ovvv = 0
wt2ovvv += 0.250000000 * np.einsum("iajk,jkbc->iabc",vmat_ovoo,t2,optimize="optimal")
wt2ovvv += 1.000000000 * np.einsum("bdja,ijcd->ibac",vmat_vvov,t2,optimize="optimal")
print("wt2ovvv = ",wt2ovvv.shape)

wt2vvoo = 0
wt2vvoo += 0.125000000 * np.einsum("klij,abkl->abij",vmat_oooo,t2.transpose(),optimize="optimal")
wt2vvoo += -1.000000000 * np.einsum("kaic,bcjk->abij",vmat_ovov,t2.transpose(),optimize="optimal")
wt2vvoo += 0.125000000 * np.einsum("abcd,cdij->abij",vmat_vvvv,t2.transpose(),optimize="optimal")
print("wt2vvoo = ",wt2vvoo.shape)

wt2vvov = 0
wt2vvov += 0.250000000 * np.einsum("jkia,bcjk->bcia",vmat_ooov,t2.transpose(),optimize="optimal")
wt2vvov += 1.000000000 * np.einsum("jbad,cdij->bcia",vmat_ovvv,t2.transpose(),optimize="optimal")
print("wt2vvov = ",wt2vvov.shape)

wt2vvvv = 0
wt2vvvv += 0.125000000 * np.einsum("abij,ijcd->abcd",vmat_vvoo,t2,optimize="optimal")
wt2vvvv += 0.125000000 * np.einsum("ijab,cdij->cdab",vmat_oovv,t2.transpose(),optimize="optimal")
print("wt2vvvv = ",wt2vvvv.shape)

wt2oo = 0
wt2oo += 0.500000000 * np.einsum("abik,jkab->ji",vmat_vvoo,t2,optimize="optimal")
wt2oo += 0.500000000 * np.einsum("ikab,abjk->ij",vmat_oovv,t2.transpose(),optimize="optimal")
print("wt2oo = ",wt2oo.shape)

wt2ov = 0
wt2ov += -0.500000000 * np.einsum("ibjk,jkab->ia",vmat_ovoo,t2,optimize="optimal")
wt2ov += -0.500000000 * np.einsum("bcja,ijbc->ia",vmat_vvov,t2,optimize="optimal")
print("wt2ov = ",wt2ov.shape)

wt2vo = 0
wt2vo += -0.500000000 * np.einsum("jkib,abjk->ai",vmat_ooov,t2.transpose(),optimize="optimal")
wt2vo += -0.500000000 * np.einsum("jabc,bcij->ai",vmat_ovvv,t2.transpose(),optimize="optimal")
print("wt2vo = ",wt2vo.shape)

wt2vv = 0
wt2vv += -0.500000000 * np.einsum("acij,ijbc->ab",vmat_vvoo,t2,optimize="optimal")
wt2vv += -0.500000000 * np.einsum("ijac,bcij->ba",vmat_oovv,t2.transpose(),optimize="optimal")
print("wt2vv = ",wt2vv.shape)

wt2 = 0 
wt2 += 0.250000000 * np.einsum("abij,ijab->",vmat_vvoo,t2,optimize="optimal")
wt2 += 0.250000000 * np.einsum("ijab,abij->",vmat_oovv,t2.transpose(),optimize="optimal")
print("wt2 = ",wt2.shape)

wt2ooooov =  (4, 4, 4, 4, 4, 24)
wt2oooovv =  (4, 4, 4, 4, 24, 24)
wt2ooovvv =  (4, 4, 4, 24, 24, 24)
wt2oovooo =  (4, 4, 24, 4, 4, 4)
wt2oovoov =  (4, 4, 24, 4, 4, 24)
wt2oovovv =  (4, 4, 24, 4, 24, 24)
wt2oovvvv =  (4, 4, 24, 24, 24, 24)
wt2ovvooo =  (4, 24, 24, 4, 4, 4)
wt2ovvoov =  (4, 24, 24, 4, 4, 24)
wt2ovvovv =  (4, 24, 24, 4, 24, 24)
wt2ovvvvv =  (4, 24, 24, 24, 24, 24)
wt2vvvooo =  (24, 24, 24, 4, 4, 4)
wt2vvvoov =  (24, 24, 24, 4, 4, 24)
wt2vvvovv =  (24, 24, 24, 4, 24, 24)
wt2oooo =  (4, 4, 4, 4)
wt2ooov =  (4, 4, 4, 24)
wt2oovv =  (4, 4, 24, 24)
wt2ovoo =  (4, 24, 4, 4)
wt2ovov =  (4, 24, 4, 24)
wt2ovvv =  (4, 24, 24, 24)
wt2vvoo =  (24, 24, 4, 4)
wt2vvov =  (24, 24, 4, 24)
wt2vvvv =  (24, 24, 24, 24)
wt2oo =  (4, 4)
wt2ov =  (4, 24)
wt2vo =  (24, 4)
wt2vv =  (24, 24)
wt2 =  ()


In [32]:
# [[F,T1]T1]
# t_ov = t1 and t_vo = transpose(t1)

ft1t1oo = 0
ft1t1oo += -1.000000000 * np.einsum("ki,ja,ak->ji",fmat_oo,t1,t1.transpose(),optimize="optimal")
ft1t1oo += -1.000000000 * np.einsum("ik,ka,aj->ij",fmat_oo,t1,t1.transpose(),optimize="optimal")
ft1t1oo += 2.000000000 * np.einsum("ba,ib,aj->ij",fmat_vv,t1,t1.transpose(),optimize="optimal")
print("ft1t1oo = ",ft1t1oo.shape)

ft1t1ov = 0
ft1t1ov += -2.000000000 * np.einsum("bj,ja,ib->ia",fmat_vo,t1,t1,optimize="optimal")
ft1t1ov += -1.000000000 * np.einsum("ja,ib,bj->ia",fmat_ov,t1,t1.transpose(),optimize="optimal")
ft1t1ov += -1.000000000 * np.einsum("ib,ja,bj->ia",fmat_ov,t1,t1.transpose(),optimize="optimal")
print("ft1t1ov = ",ft1t1ov.shape)

ft1t1vo = 0
ft1t1vo += -1.000000000 * np.einsum("bi,jb,aj->ai",fmat_vo,t1,t1.transpose(),optimize="optimal")
ft1t1vo += -1.000000000 * np.einsum("aj,jb,bi->ai",fmat_vo,t1,t1.transpose(),optimize="optimal")
ft1t1vo += -2.000000000 * np.einsum("jb,bi,aj->ai",fmat_ov,t1.transpose(),t1.transpose(),optimize="optimal")
print("ft1t1vo = ",ft1t1vo.shape)

ft1t1vv = 0
ft1t1vv += 2.000000000 * np.einsum("ji,ia,bj->ba",fmat_oo,t1,t1.transpose(),optimize="optimal")
ft1t1vv += -1.000000000 * np.einsum("ca,ic,bi->ba",fmat_vv,t1,t1.transpose(),optimize="optimal")
ft1t1vv += -1.000000000 * np.einsum("ac,ib,ci->ab",fmat_vv,t1,t1.transpose(),optimize="optimal")
print("ft1t1vv = ",ft1t1vv.shape)

ft1t1 = 0
ft1t1 += -2.000000000 * np.einsum("ji,ia,aj->",fmat_oo,t1,t1.transpose(),optimize="optimal")
ft1t1 += 2.000000000 * np.einsum("ba,ib,ai->",fmat_vv,t1,t1.transpose(),optimize="optimal")
print("ft1t1 = ",ft1t1.shape)

ft1t1oo =  (4, 4)
ft1t1ov =  (4, 24)
ft1t1vo =  (24, 4)
ft1t1vv =  (24, 24)
ft1t1 =  ()


In [33]:
# [[F,T2]T1]
# t_ov = t1 and t_vo = transpose(t1)
# t_oovv = t2 and t_vvoo = transpose(t2)

ft2t1oooo = 0
ft2t1oooo += 0.500000000 * np.einsum("ai,bj,klab->klij",fmat_vo,t1.transpose(),t2,optimize="optimal")
ft2t1oooo += 0.500000000 * np.einsum("ia,jb,abkl->ijkl",fmat_ov,t1,t2.transpose(),optimize="optimal")
print("ft2t1oooo = ",ft2t1oooo.shape)

ft2t1ooov = 0
ft2t1ooov += -1.000000000 * np.einsum("il,bj,klab->ikja",fmat_oo,t1.transpose(),t2,optimize="optimal")
ft2t1ooov += -0.500000000 * np.einsum("ba,ci,jkbc->jkia",fmat_vv,t1.transpose(),t2,optimize="optimal")
ft2t1ooov += -0.500000000 * np.einsum("cb,bi,jkac->jkia",fmat_vv,t1.transpose(),t2,optimize="optimal")
print("ft2t1ooov = ",ft2t1ooov.shape)

ft2t1oovv = 0
ft2t1oovv += 0.500000000 * np.einsum("ck,ka,ijbc->ijab",fmat_vo,t1,t2,optimize="optimal")
ft2t1oovv += 0.500000000 * np.einsum("ck,ic,jkab->ijab",fmat_vo,t1,t2,optimize="optimal")
print("ft2t1oovv = ",ft2t1oovv.shape)

ft2t1ovoo = 0
ft2t1ovoo += -1.000000000 * np.einsum("li,jb,abkl->jaik",fmat_oo,t1,t2.transpose(),optimize="optimal")
ft2t1ovoo += -0.500000000 * np.einsum("ab,ic,bcjk->iajk",fmat_vv,t1,t2.transpose(),optimize="optimal")
ft2t1ovoo += -0.500000000 * np.einsum("cb,ic,abjk->iajk",fmat_vv,t1,t2.transpose(),optimize="optimal")
print("ft2t1ovoo = ",ft2t1ovoo.shape)

ft2t1ovov = 0
ft2t1ovov += 1.000000000 * np.einsum("ci,ak,jkbc->jaib",fmat_vo,t1.transpose(),t2,optimize="optimal")
ft2t1ovov += 1.000000000 * np.einsum("ak,ci,jkbc->jaib",fmat_vo,t1.transpose(),t2,optimize="optimal")
ft2t1ovov += 1.000000000 * np.einsum("ka,ic,bcjk->ibja",fmat_ov,t1,t2.transpose(),optimize="optimal")
ft2t1ovov += 1.000000000 * np.einsum("ic,ka,bcjk->ibja",fmat_ov,t1,t2.transpose(),optimize="optimal")
print("ft2t1ovov = ",ft2t1ovov.shape)

ft2t1ovvv = 0
ft2t1ovvv += 0.500000000 * np.einsum("ij,ak,jkbc->iabc",fmat_oo,t1.transpose(),t2,optimize="optimal")
ft2t1ovvv += 0.500000000 * np.einsum("kj,ak,ijbc->iabc",fmat_oo,t1.transpose(),t2,optimize="optimal")
ft2t1ovvv += 1.000000000 * np.einsum("da,bj,ijcd->ibac",fmat_vv,t1.transpose(),t2,optimize="optimal")
print("ft2t1ovvv = ",ft2t1ovvv.shape)

ft2t1vvoo = 0
ft2t1vvoo += 0.500000000 * np.einsum("kc,ci,abjk->abij",fmat_ov,t1.transpose(),t2.transpose(),optimize="optimal")
ft2t1vvoo += 0.500000000 * np.einsum("kc,ak,bcij->abij",fmat_ov,t1.transpose(),t2.transpose(),optimize="optimal")
print("ft2t1vvoo = ",ft2t1vvoo.shape)

ft2t1vvov = 0
ft2t1vvov += 0.500000000 * np.einsum("ji,ka,bcjk->bcia",fmat_oo,t1,t2.transpose(),optimize="optimal")
ft2t1vvov += 0.500000000 * np.einsum("kj,ja,bcik->bcia",fmat_oo,t1,t2.transpose(),optimize="optimal")
ft2t1vvov += 1.000000000 * np.einsum("ad,jb,cdij->acib",fmat_vv,t1,t2.transpose(),optimize="optimal")
print("ft2t1vvov = ",ft2t1vvov.shape)

ft2t1vvvv = 0
ft2t1vvvv += 0.500000000 * np.einsum("ai,bj,ijcd->abcd",fmat_vo,t1.transpose(),t2,optimize="optimal")
ft2t1vvvv += 0.500000000 * np.einsum("ia,jb,cdij->cdab",fmat_ov,t1,t2.transpose(),optimize="optimal")
print("ft2t1vvvv = ",ft2t1vvvv.shape)

ft2t1oo = 0
ft2t1oo += 1.000000000 * np.einsum("ai,bk,jkab->ji",fmat_vo,t1.transpose(),t2,optimize="optimal")
ft2t1oo += -1.000000000 * np.einsum("ak,bi,jkab->ji",fmat_vo,t1.transpose(),t2,optimize="optimal")
ft2t1oo += 1.000000000 * np.einsum("ia,kb,abjk->ij",fmat_ov,t1,t2.transpose(),optimize="optimal")
ft2t1oo += -1.000000000 * np.einsum("ka,ib,abjk->ij",fmat_ov,t1,t2.transpose(),optimize="optimal")
print("ft2t1oo = ",ft2t1oo.shape)

ft2t1ov = 0
ft2t1ov += -1.000000000 * np.einsum("ij,bk,jkab->ia",fmat_oo,t1.transpose(),t2,optimize="optimal")
ft2t1ov += -1.000000000 * np.einsum("kj,bk,ijab->ia",fmat_oo,t1.transpose(),t2,optimize="optimal")
ft2t1ov += 1.000000000 * np.einsum("ba,cj,ijbc->ia",fmat_vv,t1.transpose(),t2,optimize="optimal")
ft2t1ov += 1.000000000 * np.einsum("cb,bj,ijac->ia",fmat_vv,t1.transpose(),t2,optimize="optimal")
print("ft2t1ov = ",ft2t1ov.shape)

ft2t1vo = 0
ft2t1vo += -1.000000000 * np.einsum("ji,kb,abjk->ai",fmat_oo,t1,t2.transpose(),optimize="optimal")
ft2t1vo += -1.000000000 * np.einsum("kj,jb,abik->ai",fmat_oo,t1,t2.transpose(),optimize="optimal")
ft2t1vo += 1.000000000 * np.einsum("ab,jc,bcij->ai",fmat_vv,t1,t2.transpose(),optimize="optimal")
ft2t1vo += 1.000000000 * np.einsum("cb,jc,abij->ai",fmat_vv,t1,t2.transpose(),optimize="optimal")
print("ft2t1vo = ",ft2t1vo.shape)

ft2t1vv = 0
ft2t1vv += -1.000000000 * np.einsum("ai,cj,ijbc->ab",fmat_vo,t1.transpose(),t2,optimize="optimal")
ft2t1vv += 1.000000000 * np.einsum("ci,aj,ijbc->ab",fmat_vo,t1.transpose(),t2,optimize="optimal")
ft2t1vv += -1.000000000 * np.einsum("ia,jc,bcij->ba",fmat_ov,t1,t2.transpose(),optimize="optimal")
ft2t1vv += 1.000000000 * np.einsum("ic,ja,bcij->ba",fmat_ov,t1,t2.transpose(),optimize="optimal")
print("ft2t1vv = ",ft2t1vv.shape)

ft2t1 = 0
ft2t1 += 1.000000000 * np.einsum("ai,bj,ijab->",fmat_vo,t1.transpose(),t2,optimize="optimal")
ft2t1 += 1.000000000 * np.einsum("ia,jb,abij->",fmat_ov,t1,t2.transpose(),optimize="optimal")
print("ft2t1 = ",ft2t1.shape)

ft2t1oooo =  (4, 4, 4, 4)
ft2t1ooov =  (4, 4, 4, 24)
ft2t1oovv =  (4, 4, 24, 24)
ft2t1ovoo =  (4, 24, 4, 4)
ft2t1ovov =  (4, 24, 4, 24)
ft2t1ovvv =  (4, 24, 24, 24)
ft2t1vvoo =  (24, 24, 4, 4)
ft2t1vvov =  (24, 24, 4, 24)
ft2t1vvvv =  (24, 24, 24, 24)
ft2t1oo =  (4, 4)
ft2t1ov =  (4, 24)
ft2t1vo =  (24, 4)
ft2t1vv =  (24, 24)
ft2t1 =  ()


In [34]:
# [[F,T1]T2]
# t_ov = t1 and t_vo = transpose(t1)
# t_oovv = t2 and t_vvoo = transpose(t2)

ft1t2ooov = 0
ft1t2ooov += 0.500000000 * np.einsum("li,bl,jkab->jkia",fmat_oo,t1.transpose(),t2,optimize="optimal")
ft1t2ooov += -0.500000000 * np.einsum("cb,bi,jkac->jkia",fmat_vv,t1.transpose(),t2,optimize="optimal")
print("ft1t2ooov = ",ft1t2ooov.shape)

ft1t2oovv = 0
ft1t2oovv += 0.500000000 * np.einsum("ck,ka,ijbc->ijab",fmat_vo,t1,t2,optimize="optimal")
ft1t2oovv += 0.500000000 * np.einsum("ck,ic,jkab->ijab",fmat_vo,t1,t2,optimize="optimal")
ft1t2oovv += 0.500000000 * np.einsum("ka,ck,ijbc->ijab",fmat_ov,t1.transpose(),t2,optimize="optimal")
ft1t2oovv += 0.500000000 * np.einsum("ic,ck,jkab->ijab",fmat_ov,t1.transpose(),t2,optimize="optimal")
print("ft1t2oovv = ",ft1t2oovv.shape)

ft1t2ovoo = 0
ft1t2ovoo += 0.500000000 * np.einsum("il,lb,abjk->iajk",fmat_oo,t1,t2.transpose(),optimize="optimal")
ft1t2ovoo += -0.500000000 * np.einsum("cb,ic,abjk->iajk",fmat_vv,t1,t2.transpose(),optimize="optimal")
print("ft1t2ovoo = ",ft1t2ovoo.shape)

ft1t2ovvv = 0
ft1t2ovvv += 0.500000000 * np.einsum("kj,ak,ijbc->iabc",fmat_oo,t1.transpose(),t2,optimize="optimal")
ft1t2ovvv += -0.500000000 * np.einsum("ad,dj,ijbc->iabc",fmat_vv,t1.transpose(),t2,optimize="optimal")
print("ft1t2ovvv = ",ft1t2ovvv.shape)

ft1t2vvoo = 0
ft1t2vvoo += 0.500000000 * np.einsum("ci,kc,abjk->abij",fmat_vo,t1,t2.transpose(),optimize="optimal")
ft1t2vvoo += 0.500000000 * np.einsum("ak,kc,bcij->abij",fmat_vo,t1,t2.transpose(),optimize="optimal")
ft1t2vvoo += 0.500000000 * np.einsum("kc,ci,abjk->abij",fmat_ov,t1.transpose(),t2.transpose(),optimize="optimal")
ft1t2vvoo += 0.500000000 * np.einsum("kc,ak,bcij->abij",fmat_ov,t1.transpose(),t2.transpose(),optimize="optimal")
print("ft1t2vvoo = ",ft1t2vvoo.shape)

ft1t2vvov = 0
ft1t2vvov += 0.500000000 * np.einsum("kj,ja,bcik->bcia",fmat_oo,t1,t2.transpose(),optimize="optimal")
ft1t2vvov += -0.500000000 * np.einsum("da,jd,bcij->bcia",fmat_vv,t1,t2.transpose(),optimize="optimal")
print("ft1t2vvov = ",ft1t2vvov.shape)

ft1t2ov = 0
ft1t2ov += -1.000000000 * np.einsum("kj,bk,ijab->ia",fmat_oo,t1.transpose(),t2,optimize="optimal")
ft1t2ov += 1.000000000 * np.einsum("cb,bj,ijac->ia",fmat_vv,t1.transpose(),t2,optimize="optimal")
print("ft1t2ov = ",ft1t2ov.shape)

ft1t2vo = 0
ft1t2vo += -1.000000000 * np.einsum("kj,jb,abik->ai",fmat_oo,t1,t2.transpose(),optimize="optimal")
ft1t2vo += 1.000000000 * np.einsum("cb,jc,abij->ai",fmat_vv,t1,t2.transpose(),optimize="optimal")
print("ft1t2vo = ",ft1t2vo.shape)

ft1t2ooov =  (4, 4, 4, 24)
ft1t2oovv =  (4, 4, 24, 24)
ft1t2ovoo =  (4, 24, 4, 4)
ft1t2ovvv =  (4, 24, 24, 24)
ft1t2vvoo =  (24, 24, 4, 4)
ft1t2vvov =  (24, 24, 4, 24)
ft1t2ov =  (4, 24)
ft1t2vo =  (24, 4)


In [35]:
# [[F,T2]T2]
# t_oovv = t2 and t_vvoo = transpose(t2)

ft2t2ooooov = 0
ft2t2ooooov += -0.250000000 * np.einsum("ib,jkac,bclm->ijklma",fmat_ov,t2,t2.transpose(),optimize="optimal")
print("ft2t2ooooov = ",ft2t2ooooov.shape)

ft2t2ooovvv = 0
ft2t2ooovvv += 0.500000000 * np.einsum("dl,ilab,jkcd->ijkabc",fmat_vo,t2,t2,optimize="optimal")
print("ft2t2ooovvv = ",ft2t2ooovvv.shape)

ft2t2oovooo = 0
ft2t2oovooo += -0.250000000 * np.einsum("bi,jkbc,aclm->jkailm",fmat_vo,t2,t2.transpose(),optimize="optimal")
print("ft2t2oovooo = ",ft2t2oovooo.shape)

ft2t2oovoov = 0
ft2t2oovoov += -0.500000000 * np.einsum("mi,jkac,bclm->jkbila",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2oovoov += -0.500000000 * np.einsum("im,jmac,bckl->ijbkla",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2oovoov += -0.250000000 * np.einsum("ca,ijcd,bdkl->ijbkla",fmat_vv,t2,t2.transpose(),optimize="optimal")
ft2t2oovoov += -0.250000000 * np.einsum("ac,ijbd,cdkl->ijaklb",fmat_vv,t2,t2.transpose(),optimize="optimal")
ft2t2oovoov += -0.500000000 * np.einsum("dc,ijad,bckl->ijbkla",fmat_vv,t2,t2.transpose(),optimize="optimal")
print("ft2t2oovoov = ",ft2t2oovoov.shape)

ft2t2oovovv = 0
ft2t2oovovv += 0.500000000 * np.einsum("la,ijbd,cdkl->ijckab",fmat_ov,t2,t2.transpose(),optimize="optimal")
ft2t2oovovv += 0.500000000 * np.einsum("id,jlab,cdkl->ijckab",fmat_ov,t2,t2.transpose(),optimize="optimal")
print("ft2t2oovovv = ",ft2t2oovovv.shape)

ft2t2ovvoov = 0
ft2t2ovvoov += 0.500000000 * np.einsum("di,jlad,bckl->jbcika",fmat_vo,t2,t2.transpose(),optimize="optimal")
ft2t2ovvoov += 0.500000000 * np.einsum("al,ilbd,cdjk->iacjkb",fmat_vo,t2,t2.transpose(),optimize="optimal")
print("ft2t2ovvoov = ",ft2t2ovvoov.shape)

ft2t2ovvovv = 0
ft2t2ovvovv += -0.250000000 * np.einsum("ki,jlab,cdkl->jcdiab",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2ovvovv += -0.250000000 * np.einsum("ik,klab,cdjl->icdjab",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2ovvovv += -0.500000000 * np.einsum("lk,ikab,cdjl->icdjab",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2ovvovv += -0.500000000 * np.einsum("ea,ikbe,cdjk->icdjab",fmat_vv,t2,t2.transpose(),optimize="optimal")
ft2t2ovvovv += -0.500000000 * np.einsum("ae,ikbc,dejk->iadjbc",fmat_vv,t2,t2.transpose(),optimize="optimal")
print("ft2t2ovvovv = ",ft2t2ovvovv.shape)

ft2t2ovvvvv = 0
ft2t2ovvvvv += -0.250000000 * np.einsum("ja,ikbc,dejk->ideabc",fmat_ov,t2,t2.transpose(),optimize="optimal")
print("ft2t2ovvvvv = ",ft2t2ovvvvv.shape)

ft2t2vvvooo = 0
ft2t2vvvooo += 0.500000000 * np.einsum("ld,adij,bckl->abcijk",fmat_ov,t2.transpose(),t2.transpose(),optimize="optimal")
print("ft2t2vvvooo = ",ft2t2vvvooo.shape)

ft2t2vvvovv = 0
ft2t2vvvovv += -0.250000000 * np.einsum("aj,jkbc,deik->adeibc",fmat_vo,t2,t2.transpose(),optimize="optimal")
print("ft2t2vvvovv = ",ft2t2vvvovv.shape)

ft2t2oooo = 0
ft2t2oooo += 0.250000000 * np.einsum("mi,jkab,ablm->jkil",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2oooo += 0.250000000 * np.einsum("im,jmab,abkl->ijkl",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2oooo += 0.500000000 * np.einsum("ba,ijbc,ackl->ijkl",fmat_vv,t2,t2.transpose(),optimize="optimal")
print("ft2t2oooo = ",ft2t2oooo.shape)

ft2t2ooov = 0
ft2t2ooov += -0.250000000 * np.einsum("la,ijbc,bckl->ijka",fmat_ov,t2,t2.transpose(),optimize="optimal")
ft2t2ooov += 1.000000000 * np.einsum("ib,jlac,bckl->ijka",fmat_ov,t2,t2.transpose(),optimize="optimal")
ft2t2ooov += 0.500000000 * np.einsum("lb,ijac,bckl->ijka",fmat_ov,t2,t2.transpose(),optimize="optimal")
print("ft2t2ooov = ",ft2t2ooov.shape)

ft2t2ovov = 0
ft2t2ovov += 1.000000000 * np.einsum("ki,jlac,bckl->jbia",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2ovov += 1.000000000 * np.einsum("ik,klac,bcjl->ibja",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2ovov += 2.000000000 * np.einsum("lk,ikac,bcjl->ibja",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2ovov += -1.000000000 * np.einsum("ca,ikcd,bdjk->ibja",fmat_vv,t2,t2.transpose(),optimize="optimal")
ft2t2ovov += -1.000000000 * np.einsum("ac,ikbd,cdjk->iajb",fmat_vv,t2,t2.transpose(),optimize="optimal")
ft2t2ovov += -2.000000000 * np.einsum("dc,ikad,bcjk->ibja",fmat_vv,t2,t2.transpose(),optimize="optimal")
print("ft2t2ovov = ",ft2t2ovov.shape)

ft2t2ovoo = 0
ft2t2ovoo += 1.000000000 * np.einsum("bi,jlbc,ackl->jaik",fmat_vo,t2,t2.transpose(),optimize="optimal")
ft2t2ovoo += -0.250000000 * np.einsum("al,ilbc,bcjk->iajk",fmat_vo,t2,t2.transpose(),optimize="optimal")
ft2t2ovoo += 0.500000000 * np.einsum("bl,ilbc,acjk->iajk",fmat_vo,t2,t2.transpose(),optimize="optimal")
print("ft2t2ovoo = ",ft2t2ovoo.shape)

ft2t2ovvv = 0
ft2t2ovvv += 1.000000000 * np.einsum("ja,ikbd,cdjk->icab",fmat_ov,t2,t2.transpose(),optimize="optimal")
ft2t2ovvv += -0.250000000 * np.einsum("id,jkab,cdjk->icab",fmat_ov,t2,t2.transpose(),optimize="optimal")
ft2t2ovvv += 0.500000000 * np.einsum("jd,ikab,cdjk->icab",fmat_ov,t2,t2.transpose(),optimize="optimal")
print("ft2t2ovvv = ",ft2t2ovvv.shape)

ft2t2vvov = 0
ft2t2vvov += -0.250000000 * np.einsum("di,jkad,bcjk->bcia",fmat_vo,t2,t2.transpose(),optimize="optimal")
ft2t2vvov += 1.000000000 * np.einsum("aj,jkbd,cdik->acib",fmat_vo,t2,t2.transpose(),optimize="optimal")
ft2t2vvov += 0.500000000 * np.einsum("dj,jkad,bcik->bcia",fmat_vo,t2,t2.transpose(),optimize="optimal")
print("ft2t2vvov = ",ft2t2vvov.shape)

ft2t2vvvv = 0
ft2t2vvvv += -0.500000000 * np.einsum("ji,ikab,cdjk->cdab",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2vvvv += -0.250000000 * np.einsum("ea,ijbe,cdij->cdab",fmat_vv,t2,t2.transpose(),optimize="optimal")
ft2t2vvvv += -0.250000000 * np.einsum("ae,ijbc,deij->adbc",fmat_vv,t2,t2.transpose(),optimize="optimal")
print("ft2t2vvvv = ",ft2t2vvvv.shape)

ft2t2oo = 0
ft2t2oo += -0.500000000 * np.einsum("ki,jlab,abkl->ji",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2oo += -0.500000000 * np.einsum("ik,klab,abjl->ij",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2oo += -1.000000000 * np.einsum("lk,ikab,abjl->ij",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2oo += 2.000000000 * np.einsum("ba,ikbc,acjk->ij",fmat_vv,t2,t2.transpose(),optimize="optimal")
print("ft2t2oo = ",ft2t2oo.shape)

ft2t2ov = 0
ft2t2ov += -0.500000000 * np.einsum("ja,ikbc,bcjk->ia",fmat_ov,t2,t2.transpose(),optimize="optimal")
ft2t2ov += -0.500000000 * np.einsum("ib,jkac,bcjk->ia",fmat_ov,t2,t2.transpose(),optimize="optimal")
ft2t2ov += 1.000000000 * np.einsum("jb,ikac,bcjk->ia",fmat_ov,t2,t2.transpose(),optimize="optimal")
print("ft2t2ov = ",ft2t2ov.shape)

ft2t2vo = 0
ft2t2vo += -0.500000000 * np.einsum("bi,jkbc,acjk->ai",fmat_vo,t2,t2.transpose(),optimize="optimal")
ft2t2vo += -0.500000000 * np.einsum("aj,jkbc,bcik->ai",fmat_vo,t2,t2.transpose(),optimize="optimal")
ft2t2vo += 1.000000000 * np.einsum("bj,jkbc,acik->ai",fmat_vo,t2,t2.transpose(),optimize="optimal")
print("ft2t2vo = ",ft2t2vo.shape)

ft2t2vv = 0
ft2t2vv += 2.000000000 * np.einsum("ji,ikac,bcjk->ba",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2vv += -0.500000000 * np.einsum("ca,ijcd,bdij->ba",fmat_vv,t2,t2.transpose(),optimize="optimal")
ft2t2vv += -0.500000000 * np.einsum("ac,ijbd,cdij->ab",fmat_vv,t2,t2.transpose(),optimize="optimal")
ft2t2vv += -1.000000000 * np.einsum("dc,ijad,bcij->ba",fmat_vv,t2,t2.transpose(),optimize="optimal")
print("ft2t2vv = ",ft2t2vv.shape)

ft2t2 = 0
ft2t2 += -1.000000000 * np.einsum("ji,ikab,abjk->",fmat_oo,t2,t2.transpose(),optimize="optimal")
ft2t2 += 1.000000000 * np.einsum("ba,ijbc,acij->",fmat_vv,t2,t2.transpose(),optimize="optimal")
print("ft2t2 = ",ft2t2.shape)

ft2t2ooooov =  (4, 4, 4, 4, 4, 24)
ft2t2ooovvv =  (4, 4, 4, 24, 24, 24)
ft2t2oovooo =  (4, 4, 24, 4, 4, 4)
ft2t2oovoov =  (4, 4, 24, 4, 4, 24)
ft2t2oovovv =  (4, 4, 24, 4, 24, 24)
ft2t2ovvoov =  (4, 24, 24, 4, 4, 24)
ft2t2ovvovv =  (4, 24, 24, 4, 24, 24)
ft2t2ovvvvv =  (4, 24, 24, 24, 24, 24)
ft2t2vvvooo =  (24, 24, 24, 4, 4, 4)
ft2t2vvvovv =  (24, 24, 24, 4, 24, 24)
ft2t2oooo =  (4, 4, 4, 4)
ft2t2ooov =  (4, 4, 4, 24)
ft2t2ovov =  (4, 24, 4, 24)
ft2t2ovoo =  (4, 24, 4, 4)
ft2t2ovvv =  (4, 24, 24, 24)
ft2t2vvov =  (24, 24, 4, 24)
ft2t2vvvv =  (24, 24, 24, 24)
ft2t2oo =  (4, 4)
ft2t2ov =  (4, 24)
ft2t2vo =  (24, 4)
ft2t2vv =  (24, 24)
ft2t2 =  ()
